In [3]:
# %% [markdown]
# # 🏠 Airbnb Price & Availability Prediction
# Predict **nightly price** and **availability** using trained ML models.

# %%
# Install dependencies if needed
# !pip install ipywidgets plotly pandas numpy joblib

# %%
import numpy as np
import pandas as pd
import joblib
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# %%
# ==========================
# LOAD MODELS & ENCODERS
# ==========================
price_model = joblib.load("price_model_xgb.pkl")
availability_model = joblib.load("availability_model_rf.pkl")
encoders = joblib.load("price_encoders.pkl")

# %%
# ==========================
# CREATE WIDGETS
# ==========================

# Property Info
property_type_widget = widgets.Dropdown(
    options=list(encoders["property_type"].classes_),
    description='Property Type:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

room_type_widget = widgets.Dropdown(
    options=list(encoders["room_type"].classes_),
    description='Room Type:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

neighbourhood_widget = widgets.Dropdown(
    options=list(encoders["neighbourhood_cleansed"].classes_),
    description='Neighborhood:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

accommodates_widget = widgets.IntSlider(
    value=2, min=1, max=16, step=1,
    description='Guests:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

bedrooms_widget = widgets.IntText(
    value=1, min=0, max=10,
    description='Bedrooms:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

beds_widget = widgets.IntText(
    value=1, min=0, max=10,
    description='Beds:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

bathrooms_widget = widgets.FloatText(
    value=1.0, min=0.0, max=10.0, step=0.5,
    description='Bathrooms:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

# Amenities
amenities_options = [
    "Wi-Fi", "Kitchen", "Free parking", "Washer",
    "Dryer", "Air conditioning", "Heating", "TV",
    "Pool", "Gym", "Workspace", "Elevator"
]

amenities_widget = widgets.SelectMultiple(
    options=amenities_options,
    description='Amenities:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px', height='120px')
)

# Host & Booking
host_superhost_widget = widgets.RadioButtons(
    options=['Yes', 'No'],
    value='No',
    description='Superhost:',
    style={'description_width': '120px'}
)

instant_bookable_widget = widgets.RadioButtons(
    options=['Yes', 'No'],
    value='No',
    description='Instant Book:',
    style={'description_width': '120px'}
)

month_options = [
    ("January", 1), ("February", 2), ("March", 3),
    ("April", 4), ("May", 5), ("June", 6),
    ("July", 7), ("August", 8), ("September", 9),
    ("October", 10), ("November", 11), ("December", 12)
]

month_widget = widgets.Dropdown(
    options=month_options,
    value=1,
    description='Month:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

# Predict button
predict_button = widgets.Button(
    description='🔮 Predict',
    button_style='success',
    layout=widgets.Layout(width='350px', height='40px')
)

# Output area
output_area = widgets.Output()

# %%
# ==========================
# PREDICTION FUNCTION
# ==========================

def run_prediction(b):
    with output_area:
        clear_output(wait=True)

        # Get values from widgets
        property_type = property_type_widget.value
        room_type = room_type_widget.value
        neighbourhood = neighbourhood_widget.value
        accommodates = accommodates_widget.value
        bedrooms = bedrooms_widget.value
        beds = beds_widget.value
        bathrooms = bathrooms_widget.value
        amenities = amenities_widget.value
        host_superhost = host_superhost_widget.value
        instant_bookable = instant_bookable_widget.value
        month = month_widget.value

        # Base encoded features
        base_features = {
            "property_type": encoders["property_type"].transform([property_type])[0],
            "room_type": encoders["room_type"].transform([room_type])[0],
            "neighbourhood_cleansed": encoders["neighbourhood_cleansed"].transform([neighbourhood])[0],
            "accommodates": accommodates,
            "bedrooms": bedrooms,
            "beds": beds,
            "bathrooms": bathrooms,
            "amenities_count": len(amenities),
            "host_is_superhost": encoders["host_is_superhost"].transform(
                ["t" if host_superhost == "Yes" else "f"]
            )[0],
            "instant_bookable": encoders["instant_bookable"].transform(
                ["t" if instant_bookable == "Yes" else "f"]
            )[0],
            "month": month
        }

        # Price prediction
        price_input = pd.DataFrame([{
            **base_features,
            "reviews_per_month": 1.0,
            "review_scores_rating": 95.0,
            "minimum_nights": 2,
            "number_of_reviews": 10
        }])

        price_log_pred = price_model.predict(price_input)[0]
        price_pred = np.expm1(price_log_pred)

        # Availability prediction
        avail_input = pd.DataFrame([{
            **base_features,
            "price_log": price_log_pred,
            "avg_availability": 0.5,
            "reviews_per_month": 1.0
        }])

        avail_logit = availability_model.predict(avail_input)[0]
        availability_pred = 1 / (1 + np.exp(-avail_logit))

        # Month names
        month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                       "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
        month_names_full = [
            "January", "February", "March", "April", "May", "June",
            "July", "August", "September", "October", "November", "December"
        ]

        # Display results header
        display(HTML(f"""
        <div style="background: #d4edda; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
            <h3 style="color: #155724; margin: 0;">✅ Prediction Complete!</h3>
        </div>

        <h3>📊 Prediction Results</h3>
        <div style="display: flex; gap: 20px; margin-bottom: 20px;">
            <div style="background: #f8f9fa; padding: 20px; border-radius: 8px; flex: 1; text-align: center;">
                <h4 style="margin: 0; color: #666;">💰 Estimated Price</h4>
                <h2 style="margin: 10px 0; color: #FF5A5F;">${price_pred:,.2f}</h2>
                <p style="margin: 0; color: #999;">per night</p>
            </div>
            <div style="background: #f8f9fa; padding: 20px; border-radius: 8px; flex: 1; text-align: center;">
                <h4 style="margin: 0; color: #666;">📅 Expected Availability</h4>
                <h2 style="margin: 10px 0; color: #00A699;">{availability_pred * 100:.1f}%</h2>
            </div>
            <div style="background: #f8f9fa; padding: 20px; border-radius: 8px; flex: 1; text-align: center;">
                <h4 style="margin: 0; color: #666;">📆 Selected Month</h4>
                <h2 style="margin: 10px 0; color: #484848;">{month_names_full[month - 1]}</h2>
            </div>
        </div>
        """))

        # Calculate monthly trends
        monthly_prices = []
        monthly_availability = []

        for m in range(1, 13):
            monthly_features = base_features.copy()
            monthly_features["month"] = m

            monthly_price_input = pd.DataFrame([{
                **monthly_features,
                "reviews_per_month": 1.0,
                "review_scores_rating": 95.0,
                "minimum_nights": 2,
                "number_of_reviews": 10
            }])

            monthly_log_pred = price_model.predict(monthly_price_input)[0]
            monthly_prices.append(np.expm1(monthly_log_pred))

            monthly_avail_input = pd.DataFrame([{
                **monthly_features,
                "price_log": monthly_log_pred,
                "avg_availability": 0.5,
                "reviews_per_month": 1.0
            }])

            monthly_avail_logit = availability_model.predict(monthly_avail_input)[0]
            monthly_availability.append(1 / (1 + np.exp(-monthly_avail_logit)) * 100)

        # Create side-by-side charts
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('💰 Price by Month', '📅 Availability by Month')
        )

        # Price chart
        fig.add_trace(
            go.Scatter(
                x=month_names, y=monthly_prices,
                mode='lines+markers',
                name='Price',
                line=dict(color='#FF5A5F', width=3),
                marker=dict(size=8),
                hovertemplate='%{x}<br>Price: $%{y:.2f}<extra></extra>'
            ),
            row=1, col=1
        )

        # Highlight selected month on price
        fig.add_trace(
            go.Scatter(
                x=[month_names[month - 1]],
                y=[monthly_prices[month - 1]],
                mode='markers',
                name='Selected',
                marker=dict(size=14, color='#FF5A5F', symbol='star',
                           line=dict(width=2, color='white')),
                showlegend=False
            ),
            row=1, col=1
        )

        # Availability chart
        fig.add_trace(
            go.Scatter(
                x=month_names, y=monthly_availability,
                mode='lines+markers',
                name='Availability',
                line=dict(color='#00A699', width=3),
                marker=dict(size=8),
                hovertemplate='%{x}<br>Availability: %{y:.1f}%<extra></extra>'
            ),
            row=1, col=2
        )

        # Highlight selected month on availability
        fig.add_trace(
            go.Scatter(
                x=[month_names[month - 1]],
                y=[monthly_availability[month - 1]],
                mode='markers',
                name='Selected',
                marker=dict(size=14, color='#00A699', symbol='star',
                           line=dict(width=2, color='white')),
                showlegend=False
            ),
            row=1, col=2
        )

        fig.update_layout(
            height=400,
            showlegend=False,
            hovermode='x unified'
        )

        fig.update_yaxes(title_text='Price ($)', row=1, col=1)
        fig.update_yaxes(title_text='Availability (%)', row=1, col=2)

        display(HTML("<h3>📈 Monthly Price & Availability Trends</h3>"))
        fig.show()

        # Insights
        min_price_idx = np.argmin(monthly_prices)
        max_price_idx = np.argmax(monthly_prices)
        min_avail_idx = np.argmin(monthly_availability)
        max_avail_idx = np.argmax(monthly_availability)

        display(HTML(f"""
        <h3>💡 Insights</h3>
        <div style="display: flex; gap: 20px;">
            <div style="flex: 1;">
                <h4>Price Insights</h4>
                <ul>
                    <li>📉 Lowest: <strong>${min(monthly_prices):,.2f}</strong> in {month_names_full[min_price_idx]}</li>
                    <li>📈 Highest: <strong>${max(monthly_prices):,.2f}</strong> in {month_names_full[max_price_idx]}</li>
                    <li>📊 Range: <strong>${max(monthly_prices) - min(monthly_prices):,.2f}</strong></li>
                </ul>
            </div>
            <div style="flex: 1;">
                <h4>Availability Insights</h4>
                <ul>
                    <li>🔥 Busiest: <strong>{min(monthly_availability):.1f}%</strong> available in {month_names_full[min_avail_idx]}</li>
                    <li>🌙 Slowest: <strong>{max(monthly_availability):.1f}%</strong> available in {month_names_full[max_avail_idx]}</li>
                </ul>
            </div>
        </div>

        <h3>🎯 Recommendations</h3>
        <div style="display: flex; gap: 20px;">
            <div style="background: #cce5ff; padding: 15px; border-radius: 8px; flex: 1;">
                <strong>Best month for revenue:</strong> {month_names_full[max_price_idx]}<br>
                <small>Highest predicted nightly rate.</small>
            </div>
            <div style="background: #cce5ff; padding: 15px; border-radius: 8px; flex: 1;">
                <strong>Peak demand month:</strong> {month_names_full[min_avail_idx]}<br>
                <small>Lowest availability indicates highest booking potential.</small>
            </div>
        </div>

        <br>
        <p style="color: #666; font-size: 12px;">Predictions are based on historical Airbnb New Brunswick data.</p>
        """))

        # Data table
        display(HTML("<h3>📋 Full Monthly Data</h3>"))
        chart_data = pd.DataFrame({
            "Month": month_names_full,
            "Price ($)": [f"${p:,.2f}" for p in monthly_prices],
            "Availability (%)": [f"{a:.1f}%" for a in monthly_availability]
        })
        display(chart_data)

# Connect button to function
predict_button.on_click(run_prediction)

# %%
# ==========================
# DISPLAY UI
# ==========================

display(HTML("<h2>🏠 Property Details</h2>"))

# Property info section
display(HTML("<h4>🏡 Property Info</h4>"))
display(widgets.VBox([
    property_type_widget,
    room_type_widget,
    neighbourhood_widget,
    accommodates_widget,
    bedrooms_widget,
    beds_widget,
    bathrooms_widget
]))

# Amenities section
display(HTML("<h4>🧺 Amenities</h4>"))
display(HTML("<small>Hold Ctrl/Cmd to select multiple</small>"))
display(amenities_widget)

# Host & Booking section
display(HTML("<h4>👤 Host & Booking</h4>"))
display(widgets.HBox([host_superhost_widget, instant_bookable_widget]))
display(month_widget)

# Predict button
display(HTML("<br>"))
display(predict_button)

# Output area
display(HTML("<hr>"))
display(output_area)

SelectMultiple(description='Amenities:', layout=Layout(height='120px', width='350px'), options=('Wi-Fi', 'Kitc…

Dropdown(description='Month:', layout=Layout(width='350px'), options=(('January', 1), ('February', 2), ('March…

Button(button_style='success', description='🔮 Predict', layout=Layout(height='40px', width='350px'), style=But…

Output()

In [ ]:
pip install ipywidgets

In [ ]:
jupyter nbextension enable --py widgetsnbextension